In [7]:
import time
import pyvisa
import logging
import functools
import numpy as np

class ESP8266Accelerometer:
    def __init__(self, time_offset, COM_port):
        self.time_offset = time_offset
        self.rm = pyvisa.ResourceManager()

        try:
            self.instr = self.rm.open_resource(COM_port)
            self.instr.parity = pyvisa.constants.Parity.none
            self.instr.data_bits = 8
            self.instr.write_termination = '\r\n'
            self.instr.read_termination = '\r\n'
            self.instr.baud_rate = 9600
            self.verification_string = "True"
        except pyvisa.errors.VisaIOError:
            self.verification_string = "False"
            self.instr = False
            return

        # shape and type of the array of returned data
        self.dtype = 'f4'
        self.shape = (3, )

        self.warnings = []

    def __enter__(self):
        return self

    def __exit__(self, *exc):
        if self.instr:
            self.instr.close()

    def ReadValue(self):
        values = self.instr.query('m').split(',')
        t = time.time() - self.time_offset
        values = [int(val) for val in values]
        return [t]+values

In [166]:
import h5py

def create_hdf_grp(fname, grp_name, attrs):
    with h5py.File(fname, 'a') as f:
        grp = f.create_group(grp_name)
        for k,v in attrs.items():
            grp.attrs[k] = v

def create_hdf_dset(fname, grp_name, dset_name, dtype, attrs):
    with h5py.File(fname, 'a') as f:
        grp = f[grp_name]
        dset = grp.create_dataset(dset_name, (0,), dtype = dtype, maxshape = (None,))
        for k,v in attrs.items():
            dset.attrs[k] = v 

def save_to_hdf(fname, grp_name, dset_name, data, dtype):
    with h5py.File(fname, 'a') as f:
        dset = f[grp_name][dset_name]
        dset.resize(dset.shape[0]+d.shape[0], axis = 0)
        dset[-data.shape[0]:] = data

In [167]:
%matplotlib ipympl
# %matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
import time

def plot_data(ax, x,y, colors=['C0']):
    if ax.lines:
        for line in ax.lines:
            line.set_xdata(x)
            line.set_ydata(y)
    else:
        for color in colors:
            ax.plot(x, y, color)
    fig.canvas.draw()

fig, ax = plt.subplots()
fig.canvas.layout.height = '600px'
fig.canvas.layout.width = '1000px'

ax.set_xlabel('time')
ax.set_ylabel('accelerometer')
ax.set_xlim(0,1)
ax.set_ylim(0,1);
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [168]:
from tqdm import tqdm_notebook as tqdm

nr_measurements = 5
nr_sections = 33*5

vib = ESP8266Accelerometer(time.time(), 'COM5')

conv = 4*9.81/(2**14 -1)

fname = 'vibration_measurement.hdf5'
run_name = 'measurement 1'
description = "Measurement of vibration in SPL7 next to laser table adjacent to beam source "+\
              "with an MMA8451 sensor"
names = "time, x acceleration, y acceleration, z acceleration".split(",")
units = "s, m/s^2, m/s^2, m/s^2".split(",")
dtypes = ['float', 'float', 'float', 'float']

attrs_measurement = { "description": description}
attrs = { 'units': f"{dict((name, unit) for name, unit in zip(names, units))}"}

data_type = np.dtype([(name.strip(""), eval(ty)) for name, ty in zip(names, dtypes)])

create_hdf_grp(fname, run_name, attrs_measurement)
create_hdf_dset(fname, run_name, "MMA8451", data_type, attrs = attrs)

for _ in tqdm(range(nr_measurements)):
    d = np.empty([nr_sections,4])
    for i in range(nr_sections):
        d[i,:] = np.array(vib.ReadValue())
    d[:,1:] *= conv
    dhdf = np.array([tuple(di) for di in list(d)], dtype = data_type)
    norm = np.linalg.norm(d[:,1:], axis = 1)
    offset = norm.ptp()*0.01
    ax.set_xlim([np.min(d[:,0]), np.max(d[:,0])])
    ax.set_ylim([norm.min()-offset, norm.max()+offset]);
    plot_data(ax, d[:,0], norm)
    save_to_hdf(fname, run_name, 'MMA8451', dhdf, data_type)